In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import umap
from sklearn.impute import SimpleImputer
from support import convert_date
from timeseriesdataset import TimeSeriesDataset
from torch.utils.data import Dataset, DataLoader
from timeseriestransformers import *
import joblib
[train_dataset,test_dataset,data_umap_with_targets,scaler]=joblib.load('alldata.unknown')
num_features = data_umap_with_targets.shape[1]
model_dim = 128
num_heads = 4
num_layers = 3

seq_len = 24
pred_len = 16
# target_features = list(range(-y.shape[1], 0))  # Indices of target features in the transformed data

output_dim = 5

model = TimeSeriesTransformer(num_features, model_dim, num_heads, num_layers, pred_len, output_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epoch_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)


# Load the saved model's state dictionary
PATH = 'model.pth'
model.load_state_dict(torch.load(PATH,map_location=torch.device('cpu')))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.eval()
test_loss = 0
# Single sample prediction (for a specific sequence from the test set)
src, tgt = test_dataset[500]  # Retrieve a single sequence (for example, the first sequence in the test set)
src = src.unsqueeze(0)  # Add batch dimension
tgt = tgt.unsqueeze(0)  # Add batch dimension

with torch.no_grad():
    batch_size = src.size(0)
    pred_len = tgt.size(1)
    output_dim = tgt.size(2)

    # Initialize the prediction sequence with zeros
    tgt_pred = torch.zeros(batch_size, 1, output_dim).to(src.device)

    # Autoregressive prediction loop (predict each time step sequentially)
    for _ in range(pred_len):
        output = model(src, tgt_pred)
        next_pred = output[:, -1, :].unsqueeze(1)  # Get the prediction for the next time step
        tgt_pred = torch.cat((tgt_pred, next_pred), dim=1)  # Append the next prediction

# Remove the first zero initialization step
tgt_pred = tgt_pred[:, 1:, :]
# Compute loss between predicted and actual values
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss = criterion(tgt_pred, tgt)
test_loss += loss.item()

# Inverse scaling of predictions and true values
# Assuming your data was reshaped before scaling, you might need to reshape it back
tgt_pred_np = tgt_pred.cpu().numpy().reshape(-1, output_dim)
# tgt_np = tgt.cpu().numpy().reshape(-1, output_dim)

tgt_pred_rescaledA = scaler.inverse_transform(np.hstack((torch.zeros(16,74),tgt_pred_np)))  # Inverse scale predictions
tgt_pred_rescaled=tgt_pred_rescaledA[:,-5:]
WS=np.sqrt(tgt_pred_rescaled[:,1]**2)+(tgt_pred_rescaled[:,2]**2)
WD=np.degrees(np.arctan(tgt_pred_rescaled[:,2],tgt_pred_rescaled[:,1]))

final=np.hstack((tgt_pred_rescaled[:,0:1],WD.reshape(-1,1),WS.reshape(-1,1),tgt_pred_rescaled[:,3:]))

c:\Users\vikra\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\vikra\AppData\Local\Temp\ipykernel_26928\2590236175.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitra